# Пакеты

In [1]:
from typing import List, Any
!pip install augmentex

In [2]:
import pandas as pd
from enum import Enum
from augmentex import CharAug
import random

# Загрузка данных из файла

In [3]:
df = pd.read_csv("places_address_elements.csv")
df = df.fillna('')

Удаляем города федерального значения.

In [4]:
condition_name = df.region_name == df.city_name
condition_type = df.region_type_name == df.city_type_name
df.loc[condition_name&condition_type, ['city_type_name', 'city_name', 'city_type_name_short']] = ''

In [5]:
df

,region_name,region_type_name,region_type_name_short,area_name,area_type_name,area_type_name_short,city_name,city_type_name,city_type_name_short,territory_name,territory_type_name,territory_type_name_short,street_name,street_type_name,street_type_name_short
0,Адыгея,Республика,Респ,,,,,,,,,,,,
1,Башкортостан,Республика,Респ,,,,,,,,,,,,
2,Алтай,Республика,Респ,,,,,,,,,,,,
3,Кабардино-Балкарская,Республика,Респ,,,,,,,,,,,,
4,Калмыкия,Республика,Респ,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519314,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Малиновая,Улица,ул.
1519315,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Персиковая,Улица,ул.
1519316,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Яблоневая,Улица,ул.
1519317,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Садовая,Улица,ул.


# Вспомогательные классы
## Словарь ElementType 

In [6]:

class ElementType(Enum):
    OTHER = 0
    REGION=1
    REGION_TYPE=2
    AREA=3
    AREA_TYPE=4
    TERRITORY=5
    TERRITORY_TYPE=6
    CITY=7
    CITY_TYPE=8
    STREET=9
    STREET_TYPE=10
    DELIMITER=11
    
    def getTyped(self):
        if self == ElementType.REGION:
            return ElementType.REGION_TYPE
        elif self == ElementType.AREA:
            return ElementType.AREA_TYPE
        elif self == ElementType.TERRITORY:
            return ElementType.TERRITORY_TYPE
        elif self == ElementType.CITY:
            return ElementType.CITY_TYPE
        elif self == ElementType.STREET:
            return ElementType.STREET_TYPE
        else:
            return self
        


## Класс TypedWord
Проставим каждому токену тип. Для этого введем класс TypedWord

In [7]:
class TypedWord:
    word = ''
    type = ElementType.OTHER
    def __init__(self, word:str, type:ElementType):
        self.word = word
        self.type = type
    
    def __repr__(self):
        return self.word + ":" + self.type.name

# Аугментация

## Представление набора адресо-образующих элементов в виде списка TypedWord

In [8]:
def to_typed_word_list(input , elementType :ElementType  ) -> list[TypedWord]:
    words = list(filter(('').__ne__, input.split(' ')))
    res = list()
    for word in words:
        res.append(TypedWord(word, elementType))
    return res


def typed_word_list_for_type(elements: dict, 
                             elementType :ElementType , 
                             type_first_proba=0.75, 
                             comma_sep_proba = 0.75, 
                             short_type_proba=0.5) -> list[TypedWord]:
    res: list[TypedWord] = []
    delimiter = TypedWord(',', ElementType.DELIMITER)
    base = elementType.name.lower()
    base_name = base+'_name'
    base_type_name = base+'_type_name'
    base_type_name_short = base+'_type_name_short'
    if elements[base_name]=='':
        return res
    if short_type_proba> random.random():
        type_column = base_type_name_short
    else:
        type_column = base_type_name

    if type_first_proba>random.random():
        res+= to_typed_word_list(elements[type_column], elementType.getTyped())
        res+= to_typed_word_list(elements[base_name], elementType)
    else:
        res+= (to_typed_word_list(elements[base_name], elementType))
        res+= to_typed_word_list(elements[type_column], elementType.getTyped())

    if comma_sep_proba>random.random():
        res+= [delimiter]
    return res


def convert_address_elements(elements: dict, type_first_proba=0.75, comma_sep_proba=0.75, short_type_proba=0.5):
    res: list[TypedWord] = []
    res += typed_word_list_for_type(elements, ElementType.REGION, type_first_proba=type_first_proba,
                                    comma_sep_proba=comma_sep_proba, short_type_proba=short_type_proba)
    res += typed_word_list_for_type(elements, ElementType.AREA, type_first_proba=type_first_proba,
                                    comma_sep_proba=comma_sep_proba, short_type_proba=short_type_proba)
    res += typed_word_list_for_type(elements, ElementType.CITY, type_first_proba=type_first_proba,
                                    comma_sep_proba=comma_sep_proba, short_type_proba=short_type_proba) 
    res += typed_word_list_for_type(elements, ElementType.TERRITORY, type_first_proba=type_first_proba,
                                    comma_sep_proba=comma_sep_proba, short_type_proba=short_type_proba)
    res += typed_word_list_for_type(elements, ElementType.STREET, type_first_proba=type_first_proba,
                                    comma_sep_proba=comma_sep_proba, short_type_proba=short_type_proba)
    
    if res[-1].type == ElementType.DELIMITER:
       res.pop()

    return res


elements = df.sample(random_state=442).to_dict('records')
convert_address_elements(elements[0], type_first_proba=1, comma_sep_proba=1, short_type_proba=1)

[обл:REGION_TYPE,
 Нижегородская:REGION,
 ,:DELIMITER,
 р-н:AREA_TYPE,
 Сосновский:AREA,
 ,:DELIMITER,
 д:CITY_TYPE,
 Волчиха:CITY,
 ,:DELIMITER,
 ул:STREET_TYPE,
 Пушкина:STREET]

In [9]:
sample_typed_words = convert_address_elements(df.sample(random_state=0).to_dict('records')[0])
sample_typed_words

[Край:REGION_TYPE,
 Приморский:REGION,
 ,:DELIMITER,
 р-н:AREA_TYPE,
 Черниговский:AREA,
 ,:DELIMITER,
 Сибирцево:CITY,
 Поселок:CITY_TYPE,
 городского:CITY_TYPE,
 типа:CITY_TYPE,
 ,:DELIMITER,
 Гаражно-строительный:TERRITORY_TYPE,
 кооп.:TERRITORY_TYPE,
 33г/24:TERRITORY]

In [10]:
char_aug = CharAug(
    unit_prob=0.05, # Percentage of the phrase to which augmentations will be applied
    min_aug=0, # Minimum number of augmentations
    max_aug=3, # Maximum number of augmentations
    mult_num=3, # Maximum number of repetitions of characters (only for the multiply method)
    random_seed=42,
    lang="rus", # supports: "rus", "eng"
    platform="pc", # supports: "pc", "mobile"
)


In [11]:
text = 'Автодорога Москва-Казань'
char_aug.augment(text=text)

'Автодорога Москва-Казыань'

Обратная задача. Представление списка TypedWords в виде списка слов или списка типов адресо-образующих элементов

In [12]:
def typed_words_to_words(input:list):
    words = []
    for element in input:
        words.append(element.word)
    return words    

In [13]:
def typed_words_to_type_codes(input:list):
    type_codes = []
    for element in input:
        type_codes.append(element.type.value)
    return type_codes    

# Генерация нового датасета
## Новые колонки - tokens и classes

In [ ]:
df['typed_words'] = df.apply(convert_address_elements, axis=1)
df['tokens'] = df.apply(lambda row: typed_words_to_words(row.typed_words),  axis=1)
df['classes'] = df.apply(lambda row: typed_words_to_type_codes(row.typed_words),  axis=1)

## Конструирование нового dataframe 

In [ ]:
df_token_with_classes = df[['tokens','classes']]

## Сохранение на диск

In [ ]:
df_token_with_classes.to_pickle('./df_token_with_classes.pkl')

In [ ]:
!zip archive_pkl.zip ./df_token_with_classes.pkl  